In [1]:
import numpy as np
from datasetUpload_v2p8 import scicatBam, scicatScatter
import datetime
from pathlib import Path
import pandas
import json
# from xraydb import material_mu
import xraydb
import chempy
from SAXSClasses import readLog  # reads our logbooks
import urllib

get some entries that we can start flagging

simple flags are:
  - transmission values, 
  - beam stability 

For this we need the "raw" structurized files, not the processed files.

Processed flags would be:
  - many negative datapoints
  - uncertainties of the same order of magnitude as the data
  - multiple scattering flag

In [2]:
scb = scicatBam()

In [3]:
entries = scb.getEntries(url = scb.baseurl + "datasets", whereDict = {"datasetName": {"like": "20201222*"}, "type": "raw"})
scb.getToken(username="admin", password="X")
for entry in entries:
    # clean up duplicate datablocks
    
    # get datablocks associated with entry: 
    dblockentries = scb.getEntries(url = scb.baseurl + "origdatablocks", whereDict = {"datasetId": str(entry['pid'])})
    prevFile = ''
    for entry in dblockentries:
        if entry['dataFileList'][0]['path'] == prevFile:
            # delete duplicate
            url = scb.baseurl + f"origdatablocks/{urllib.parse.quote_plus(entry['id'])}"
            r = scb.sendToSciCat(url, {}, cmd="delete")
        prevFile = entry['dataFileList'][0]['path']
    # filepath = Path(entry['sourceFolder']).glob()

In [4]:
pwd

'y:\\Measurements\\SAXS002'

In [5]:
# Now we get the datablocks again, but this time hopefully just the one.
entries = scb.getEntries(url = scb.baseurl + "datasets", whereDict = {"datasetName": {"like": "20201222*"}, "type": "raw"})
scb.getToken(username="ingestor", password="aman")
for entry in entries:
    # get datablock associated with entry: 
    dblockentries = scb.getEntries(url = scb.baseurl + "origdatablocks", whereDict = {"datasetId": str(entry['pid'])})
    for entry in dblockentries:
        filename = Path(entry['dataFileList'][0]['path'])
        # cut away the first three elements from the path as this is too much to run in TESTING mode on my laptop in windows. 
        relFilename = filename.relative_to(*filename.parts[:5])
        transmission = scb.h5Get(relFilename, '/entry1/sample/transmission', leaveAsArray = True)
        print(transmission)

    # filepath = Path(entry['sourceFolder']).glob()

[[0.00016223]
 [0.00015714]
 [0.0001501 ]
 [0.00014908]
 [0.00014939]]
[[0.38029029]
 [0.38080692]
 [0.38046145]
 [0.37992926]
 [0.37977243]]
[[0.31629427]
 [0.31635483]
 [0.31665659]
 [0.31581542]
 [0.31666199]
 [0.3146161 ]
 [0.31733455]
 [0.31631609]
 [0.31700451]
 [0.31820633]
 [0.31541895]
 [0.31673151]
 [0.31512952]
 [0.31696718]
 [0.31643418]
 [0.31571655]
 [0.31555058]
 [0.31861929]]
[[0.42078159]
 [0.42060247]
 [0.42085872]
 [0.42063569]
 [0.42209701]]
[[0.00073976]
 [0.00074646]
 [0.00076324]
 [0.00072051]
 [0.0007012 ]
 [0.00075876]
 [0.00070373]
 [0.00069904]
 [0.00072503]
 [0.00074811]
 [0.00070136]
 [0.00075711]
 [0.00069887]
 [0.00069957]
 [0.00068378]
 [0.00071275]
 [0.00069649]
 [0.00068865]]
[[0.9157831 ]
 [0.92343761]
 [0.92230551]
 [0.92145174]
 [0.91646009]
 [0.91671317]
 [0.91597536]
 [0.92081426]]
[[0.0001439 ]
 [0.00015295]
 [0.00016201]
 [0.00014647]
 [0.00016442]
 [0.00014017]
 [0.00014212]
 [0.00016683]]
[[0.31280984]
 [0.31274694]
 [0.31316424]
 [0.31256592]

In [7]:
relFilename

WindowsPath('data/2020/20201222/20201222_11_expanded_stacked.nxs')

In [47]:
scb.getToken(username="admin", password="X")
# get datablocks associated with entry: 
dblockentries = scb.getEntries(url = scb.baseurl + "origdatablocks", whereDict = {"datasetId": str(entry['pid'])})
prevFile = ''
for entry in dblockentries:
    if entry['dataFileList'][0]['path'] == prevFile:
        # delete duplicate
        url = scb.baseurl + f"origdatablocks/{urllib.parse.quote_plus(entry['id'])}"
        r = scb.sendToSciCat(url, {}, cmd="delete")
    prevFile = entry['dataFileList'][0]['path']

KeyError: 'pid'

In [48]:
dblockentries

[{'id': '5ff5c67323e1b79ff1a1f41f',
  'size': 35452997,
  'dataFileList': [{'path': '/mnt/vsi-db/Measurements/SAXS002/data/2020/20201222/20201222_41_expanded_stacked.nxs',
    'size': 35452997,
    'time': '2021-01-06T15:17:18.566Z',
    'chk': '',
    'uid': '0',
    'gid': '100',
    'perm': '33200'}],
  'ownerGroup': 'BAM 6.5',
  'accessGroups': ['BAM', 'BAM 6.5'],
  'createdBy': 'datasetUpload',
  'updatedBy': 'archiveManager',
  'datasetId': '20.500.12269/04a88c79-ce92-4a0a-860d-1cf4b39aa9d7',
  'createdAt': '2021-01-06T14:17:23.678Z',
  'updatedAt': '2021-01-06T14:17:23.704Z'},
 {'id': '5ff5cd0d23e1b75f5fa1f481',
  'size': 35452997,
  'dataFileList': [{'path': '/mnt/vsi-db/Measurements/SAXS002/data/2020/20201222/20201222_41_expanded_stacked.nxs',
    'size': 35452997,
    'time': '2021-01-06T15:45:27.942Z',
    'chk': '',
    'uid': '0',
    'gid': '100',
    'perm': '33200'}],
  'ownerGroup': 'BAM 6.5',
  'accessGroups': ['BAM', 'BAM 6.5'],
  'createdBy': 'datasetUpload',
  'upd

In [20]:
# supposed to be this, but not in this notebook, only in the eventual script:
list(Path(entry['sourceFolder']).glob(f"{entry['datasetName'].replace('-','_')}_expanded_stacked.nxs"))
# for testing:
list(Path('data/2020/20201222').glob(f"{entry['datasetName'].replace('-','_')}_expanded_stacked.nxs"))

[WindowsPath('data/2020/20201222/20201222_11_expanded_stacked.nxs')]

In [19]:
Path(entry['sourceFolder'])

WindowsPath('/mnt/vsi-db/Measurements/SAXS002/data/2020/20201222')

In [12]:
for entry in entries:
    filepath = Path(entry['sourceFolder']).glob()

0.0001536866888011287
0.38025206868675293
0.3164309358426209
0.4209939457143377
0.0007188778471223053
0.9191176050616137
0.00015216872778154973
0.31258224442929794
0.3052944708983854
0.0009495819819342119
0.409867984484767
0.3616492876602569
0.36843500975795673
0.30506356267459944
0.9190264235697178
0.9212995966431098
0.3727186921671007
0.0012696926604173215
0.0009580191739722599
0.9190496104007105
0.00044751448880083204
0.9896392425835323
0.3789571630299761
0.9891657936999861
0.00010094306836652182
0.0005212729978221488
0.2985800929775416
0.37224062161532895
0.9913883518932015
0.9893823190904854
0.2988045223194992
0.001024568856685529
0.00012863932235041998
0.41811857962210797
0.0001525590348209456
0.0004494533746370524
0.0014548061875492085
0.9219918069985779
0.9886884623431588
0.3179050031591337
0.3180598169045634
0.29323572014653143
0.4177667629641813
0.30920846920526135
0.3104178997786734
0.4175646007634069
0.00101751157136079
0.3005924988897899
0.4201657752944728
0.00147233241870

In [10]:
entry

{'pid': '20.500.12269/ef6c8632-cdb5-48df-8e19-3cf6658f4429',
 'owner': 'gsmales',
 'contactEmail': 'brian.pauw@bam.de',
 'sourceFolder': '/mnt/vsi-db/Measurements/SAXS002/data/2020/20201222',
 'size': 62651526,
 'numberOfFiles': 5,
 'creationTime': '2020-12-22T19:04:00.000Z',
 'type': 'raw',
 'datasetName': '20201222-11',
 'classification': 'IN=medium,AV=low,CO=low',
 'version': '3.1.0',
 'ownerGroup': 'BAM 6.5',
 'accessGroups': ['BAM', 'BAM 6.5'],
 'createdBy': 'archiveManager',
 'updatedBy': 'anonymous',
 'createdAt': '2021-01-06T14:15:21.554Z',
 'updatedAt': '2021-01-08T09:06:56.936Z',
 'datasetlifecycle': {'archivable': True,
  'retrievable': False,
  'publishable': False,
  'archiveRetentionTime': '2031-01-06T00:00:00.000Z',
  'dateOfPublishing': '2024-01-06T00:00:00.000Z',
  'isOnCentralDisk': True,
  'archiveStatusMessage': 'datasetCreated',
  'retrieveStatusMessage': '',
  'retrieveIntegrityCheck': False},
 'history': [],
 'instrumentId': 'SAXS002',
 'techniques': [],
 'princi